## Note:
### There are issues with Spark 3.4.0 version.
### Mount this file in Google Drive  with 'spark-3.5.0-bin-hadoop3.tgz' installed and run in Colab AI to see the results!

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!tar xf /content/drive/MyDrive/spark/spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
# !tar xf $SPARK_VERSION-bin-hadoop3.tgz
# !pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Mounted at /content/drive
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,678 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,333 kB]
Get:13 https://ppa.launchpa

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sale_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sale_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
# 2. Create a temporary view of the DataFrame.
home_sale_df.createOrReplaceTempView('home_sales')
start_time = time.time()
spark.sql("""select * from home_sales""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))


+------------------------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|id                                  |date      |date_built|price |bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+------------------------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d6-9c31-7398aa8f6089|2022-04-08|2016      |936923|4       |3        |3167       |11733   |2     |1         |76  |
|7530a2d8-1ae3-4517-9f4a-befe060c4353|2021-06-13|2013      |379628|2       |2        |2235       |14384   |1     |0         |23  |
|43de979c-0bf0-4c9f-85ef-96dc27b258d5|2019-04-12|2014      |417866|2       |2        |2127       |10575   |2     |0         |0   |
|b672c137-b88c-48bf-9f18-d0a4ac62fb8b|2019-10-16|2016      |239895|2       |2        |1631       |11149   |2     |0         |0   |
|e0726d4d-d595-4074-8283-4139a54d0d63|2022-01-08|2017      |424418|3       |2      

In [13]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
start_time = time.time()

query="""
select year(date),round(avg(price),2) avg_price
from home_sales
where bedrooms=4
group by year(date)
order by year(date)
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----------+---------+
|year(date)|avg_price|
+----------+---------+
|      2019| 300263.7|
|      2020|298353.78|
|      2021|301819.44|
|      2022|296363.88|
+----------+---------+

--- 0.7713761329650879 seconds ---


In [15]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
start_time = time.time()

query="""
select year(date),round(avg(price),2) avg_price
from home_sales
where bedrooms=3
  and bathrooms=3
group by year(date)
order by year(date)
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----------+---------+
|year(date)|avg_price|
+----------+---------+
|      2019|287287.82|
|      2020|294204.16|
|      2021|294211.46|
|      2022|292725.69|
+----------+---------+

--- 0.8047177791595459 seconds ---


In [14]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
start_time = time.time()

query="""
select year(date),round(avg(price),2) avg_price
from home_sales
where bedrooms=3
  and bathrooms=3
  and floors=2
  and sqft_living >= '2000'
group by year(date)
order by year(date)
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----------+---------+
|year(date)|avg_price|
+----------+---------+
|      2019|289859.14|
|      2020|292289.09|
|      2021|296330.96|
|      2022|290242.99|
+----------+---------+

--- 0.8226344585418701 seconds ---


In [12]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query="""
select view,round(avg(price),2) avg_price
from home_sales
group by view
having avg(price) >= 350000
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  51| 788128.21|
|  54| 798684.82|
|  69| 750537.94|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|  59|  791453.0|
|  85|1056336.74|
|  52| 733780.26|
|  71|  775651.1|
|  98|1053739.33|
|  99|1061201.42|
|  96|1017815.92|
| 100| 1026669.5|
|  70| 695865.58|
|  61| 746877.59|
|  75|1114042.94|
|  78|1080649.37|
|  89|1107839.15|
|  77|1076205.56|
+----+----------+
only showing top 20 rows

--- 1.1241049766540527 seconds ---
--- 1.124349594116211 seconds ---


In [16]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [17]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [19]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query="""
select view,round(avg(price),2) avg_price
from home_sales
group by view
having avg(price) >= 350000
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  51| 788128.21|
|  54| 798684.82|
|  69| 750537.94|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|  59|  791453.0|
|  85|1056336.74|
|  52| 733780.26|
|  71|  775651.1|
|  98|1053739.33|
|  99|1061201.42|
|  96|1017815.92|
| 100| 1026669.5|
|  70| 695865.58|
|  61| 746877.59|
|  75|1114042.94|
|  78|1080649.37|
|  89|1107839.15|
|  77|1076205.56|
+----+----------+
only showing top 20 rows

--- 0.4474050998687744 seconds ---


In [20]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sale_df.write.partitionBy("date_built").mode("overwrite").parquet("delayed_partitioned")

In [21]:
# 11. Read the formatted parquet data.
p_home_sale_df_p=spark.read.parquet('delayed_partitioned')

In [22]:
# 12. Create a temporary table for the parquet data.
p_home_sale_df_p.createOrReplaceTempView('p_parquet_delays_p')

In [23]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query="""
select view,round(avg(price),2) avg_price
from p_parquet_delays_p
group by view
having avg(price) >= 350000
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  51| 788128.21|
|  54| 798684.82|
|  69| 750537.94|
|  73| 752861.18|
|  87| 1072285.2|
|  64| 767036.67|
|  59|  791453.0|
|  85|1056336.74|
|  52| 733780.26|
|  71|  775651.1|
|  98|1053739.33|
|  99|1061201.42|
|  96|1017815.92|
| 100| 1026669.5|
|  70| 695865.58|
|  61| 746877.59|
|  75|1114042.94|
|  78|1080649.37|
|  89|1107839.15|
|  77|1076205.56|
+----+----------+
only showing top 20 rows

--- 2.004443407058716 seconds ---


In [24]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [25]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False

In [26]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached('home_sales'):
  print('home_sales is still cached')
else:
  print('all clear')

all clear
